# EDA Maze
source: https://samyzaf.com/ML/rl/qmaze.html

In [2]:
import numpy as np

In [6]:
rat_mark = 0.5      # The current rat cell will be painteg by gray 0.5

In [10]:
class Qmaze(object):
    def __init__(self, maze, rat=(0,0)):
        self._maze = np.array(maze)
        nrows, ncols = self._maze.shape
        self.target = (nrows-1, ncols-1)   # target cell where the "cheese" is
        # TODO: position of "cheese" can be chosen freely (on a free_cell)
        self.free_cells = [(r,c) for r in range(nrows) for c in range(ncols) if self._maze[r,c] == 1.0]
        # the "cheese" is not a free_cell
        self.free_cells.remove(self.target)
        if self._maze[self.target] == 0.0:
            raise Exception("Invalid maze: target cell cannot be blocked!")
        if not rat in self.free_cells:
            raise Exception("Invalid Rat Location: must sit on a free cell")
        self.reset(rat)
        
    def reset(self, rat):
        self.rat = rat
        self.maze = np.copy(self._maze)
        nrows, ncols = self.maze.shape
        row, col = rat
        self.maze[row, col] = rat_mark  # outside variable
        self.state = (row, col, 'start')
        self.min_reward = -0.5 * self.maze.size   # row * col
        self.total_reward = 0
        self.visited = set()

In [11]:
maze = [
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.]
]

np.array(maze).size

64

In [13]:
qmaze = Qmaze(maze)

In [14]:
qmaze.maze

array([[0.5, 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 1. , 0. , 1. ],
       [1. , 1. , 1. , 0. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 0. , 1. , 0. , 0. , 0. ],
       [1. , 1. , 1. , 0. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. ]])